In [3]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from aif360.algorithms.inprocessing import GerryFairClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import sklearn

import aif360
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions\
            import get_distortion_adult, get_distortion_german, get_distortion_compas

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas

from aif360.datasets import AdultDataset, BankDataset, CompasDataset, GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms import preprocessing, inprocessing, postprocessing
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.lfr import LFR
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
import copy

from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')
%load_ext jupyternotify
np.random.seed(1)

<IPython.core.display.Javascript object>

In [4]:
def run_classification_metrics(CM:ClassificationMetric):
    def f1_score(priv=None):
            numer = CM.num_true_positives(privileged=priv)
            denom = CM.num_true_positives(privileged=priv) + 0.5*float(CM.num_false_positives(privileged=priv) + CM.num_false_negatives(privileged=priv))
            return float(numer/denom)
    return np.array([
        round(CM.accuracy(), 4),
        round(f1_score(), 4),
        round(CM.theil_index(), 4),
        round(CM.false_positive_rate(privileged=False), 4),
        round(CM.false_positive_rate(privileged=True), 4),
        round(CM.false_negative_rate(privileged=False), 4),
        round(CM.false_negative_rate(privileged=True), 4),
        round(1-CM.error_rate(privileged=False), 4),
        round(1-CM.error_rate(privileged=True), 4),
        round(CM.false_discovery_rate(privileged=False), 4),
        round(CM.false_discovery_rate(privileged=True), 4),
        round(CM.false_omission_rate(privileged=False), 4),
        round(CM.false_omission_rate(privileged=True), 4),
        
        #all results
        CM.num_true_positives(),
        CM.num_true_negatives(),
        CM.num_false_positives(),
        CM.num_false_negatives(),
        
        #privileged
        CM.num_true_positives(privileged=True),
        CM.num_true_negatives(privileged=True),
        CM.num_false_positives(privileged=True),
        CM.num_false_negatives(privileged=True),
        
        #unprivileged
        CM.num_true_positives(privileged=False),
        CM.num_true_negatives(privileged=False),
        CM.num_false_positives(privileged=False),
        CM.num_false_negatives(privileged=False),
        
        round(f1_score(True), 4),
        round(f1_score(False), 4),
    ])

In [5]:
def run_binary_dataset_metrics(BLDM:BinaryLabelDatasetMetric):
    #print("Consistency: ", BLDM.consistency())
    return np.array([
        round(BLDM.base_rate(privileged=True), 4), # 1 means privileged bias
        round(BLDM.base_rate(privileged=False), 4), # 1 means unprivileged bias
        round(BLDM.consistency()[0], 4)
    ])

In [6]:
def get_model_name(model):
    if isinstance(model, sklearn.linear_model.LogisticRegression):
        return "Logistic Regression"
    if isinstance(model, sklearn.linear_model.LinearRegression):
        return "Linear Regression"
    if isinstance(model, sklearn.ensemble.BaggingClassifier):
        return "Meta Classifier"
    
    if isinstance(model, preprocessing.DisparateImpactRemover):
        return "DIR"
    if isinstance(model, preprocessing.LFR):
        return "LFR"
    if isinstance(model, preprocessing.OptimPreproc):
        return "OP"
    if isinstance(model, preprocessing.Reweighing):
        return "RW"
    
    if isinstance(model, inprocessing.PrejudiceRemover):
        return "PR"
    if isinstance(model, inprocessing.AdversarialDebiasing):
        return "AD"
    if isinstance(model, inprocessing.ARTClassifier):
        return "ARTC"
    if isinstance(model, inprocessing.ExponentiatedGradientReduction):
        return "EGR"
    if isinstance(model, inprocessing.GerryFairClassifier):
        return "GFC"
    if isinstance(model, inprocessing.GridSearchReduction):
        return "GSR"
    if isinstance(model, inprocessing.MetaFairClassifier):
        return "MFC"
    
    if isinstance(model, postprocessing.EqOddsPostprocessing):
        return "EOP"
    if isinstance(model, postprocessing.CalibratedEqOddsPostprocessing):
        return "CEOP"
    if isinstance(model, postprocessing.RejectOptionClassification):
        return "ROC"
    
    return "None"

In [7]:
def get_dataset_name(dataset):
    if isinstance(dataset, aif360.datasets.german_dataset.GermanDataset):
        return "German Dataset"
    if isinstance(dataset, aif360.datasets.adult_dataset.AdultDataset):
        return "Adult Dataset"
    if isinstance(dataset, aif360.datasets.bank_dataset.BankDataset):
        return "Bank Dataset"
    if isinstance(dataset, aif360.datasets.compas_dataset.CompasDataset):
        return "Compas Dataset"

In [49]:
# get specialized dataset for Optimized Preprocessing
def get_OP_dataset(dataset_name):
    dataset = None
    if dataset_name=="bank":
        return
    if dataset_name=="adult":
        dataset = load_preproc_data_adult(['sex'])
    elif dataset_name=="compas":
        dataset = load_preproc_data_compas(['race'])
    elif dataset_name=="german":
        dataset = load_preproc_data_german(['age'])
        dataset.labels = (dataset.labels-1.0).astype('float64')
        dataset.favorable_label = 1.0
        dataset.unfavorable_label = 0.0
        dataset.metadata['label_maps'] = [{1.0: 'Good Credit', 0.0: 'Bad Credit'}]
    return dataset

In [60]:
def get_dataset_options(dataset_name):
    optim_options = None
    if dataset_name=="adult":
        optim_options = {
        "distortion_fun": get_distortion_adult,
        "epsilon": 0.05,
        "clist": [0.99, 1.99, 2.99],
        "dlist": [.1, 0.05, 0]
        }
        pro_attr = 'sex'
        return (AdultDataset(), pro_attr, [{'sex': 1}], [{'sex': 0}], optim_options)
    elif dataset_name=="compas":
        optim_options = {
        "distortion_fun": get_distortion_compas,
        "epsilon": 0.05,
        "clist": [0.99, 1.99, 2.99],
        "dlist": [.1, 0.05, 0]
        }
        pro_attr = 'race'
        return (CompasDataset(), pro_attr, [{'race': 1}], [{'race': 0}], optim_options)
    elif dataset_name=="bank":
        pro_attr = 'age'
        return (BankDataset(protected_attribute_names=['age'],
            privileged_classes=[lambda x: x >= 25], 
            features_to_drop=['day_of_week']), pro_attr, [{'age': 1}], [{'age': 0}], None)
    elif dataset_name=="german":
        optim_options = {
            "distortion_fun": get_distortion_german,
            "epsilon": 0.1,
            "clist": [0.99, 1.99, 2.99],
            "dlist": [.1, 0.05, 0]
        }   
        pro_attr = 'age'
        label_map = {1.0: 'Good Credit', 0.0: 'Bad Credit'}
        #g = load_preproc_data_german(['age'])
        g = GermanDataset(metadata={'label_maps': [label_map]})
        g.labels = (g.labels-1.0).astype('float64')
        g.favorable_label = 1.0
        g.unfavorable_label = 0.0
        #g.metadata['label_maps'] = [label_map]

        # load_preproc_data_german(['age'])
        return (g, pro_attr, [{'age': 1}], [{'age': 0}], optim_options)

In [53]:
def execute_intervention(dataset, dataset_name, privileged_groups, unprivileged_groups, 
                 preprocessing_algo=None, inprocessing_algo=None, postprocessing_algo=None, seed=123):
    print(seed, get_model_name(preprocessing_algo), get_model_name(inprocessing_algo), get_model_name(postprocessing_algo))
    scale_orig = StandardScaler()
    dataset.features = scale_orig.fit_transform(dataset.features)
    
    # get specialized dataset for Optimized Preprocessing technique
    if get_model_name(preprocessing_algo)=="OP" and dataset_name!="bank":
        #print("Specialized function: ", dataset_name)
        dataset = get_OP_dataset(dataset_name) 
        
    #np.random.seed(seed)
    dataset_train, dataset_test = dataset.split([0.7], shuffle = True, seed=seed)

    model = sklearn.linear_model.LogisticRegression() # solver='liblinear', class_weight='balanced', 
    
    dataset_train_pred = dataset_train.copy(deepcopy=True)
    dataset_test_pred = dataset_test.copy(deepcopy=True)
            
    if preprocessing_algo is not None:
        if get_model_name(preprocessing_algo)=="DIR":
            dataset_train_pred = preprocessing_algo.fit_transform(dataset_train_pred)
            dataset_test_pred = preprocessing_algo.fit_transform(dataset_test_pred)
        elif get_model_name(preprocessing_algo) in ["OP", "LFR"]:
            preprocessing_algo.fit(dataset_train_pred)
            dataset_train_pred = preprocessing_algo.transform(dataset_train_pred)
            dataset_train_pred = dataset_train.align_datasets(dataset_train_pred)
            dataset_test_pred = preprocessing_algo.transform(dataset_test_pred)
            dataset_test_pred = dataset_test.align_datasets(dataset_test_pred)
    
    if inprocessing_algo is not None:
        inp = inprocessing_algo
        inp.fit(dataset_train_pred)
        dataset_train_pred = inp.predict(dataset_train_pred)
        dataset_test_pred = inp.predict(dataset_test_pred) 
        
        # exception for GFC 
        if get_model_name(inprocessing_algo)=="GFC":
            dataset_train_pred.scores = inp.predict(dataset_train_pred, threshold=None).labels
            dataset_test_pred.scores = inp.predict(dataset_test_pred, threshold=None).labels
            dataset_train_pred.labels = inp.predict(dataset_train_pred, threshold=0.5).labels
            dataset_test_pred.labels = inp.predict(dataset_test_pred, threshold=0.5).labels

    else:
        model.fit(dataset_train_pred.features, dataset_train_pred.labels)   # .ravel()
        fav_idx = np.where(model.classes_ == dataset_train.favorable_label)[0][0]
        dataset_train_pred.scores = model.predict_proba(dataset_train_pred.features)[:,fav_idx].reshape(-1,1) 
        dataset_train_pred.labels = model.predict(dataset_train_pred.features).reshape(-1,1) 
        dataset_test_pred.scores = model.predict_proba(dataset_test_pred.features)[:,fav_idx].reshape(-1,1) 
        dataset_test_pred.labels = model.predict(dataset_test_pred.features).reshape(-1,1)  
            
    if postprocessing_algo is not None:
        dataset_train.features = dataset_train_pred.features
        pp = postprocessing_algo
        pp = pp.fit(dataset_train, dataset_train_pred)
        dataset_test_pred = pp.predict(dataset_test_pred)

    BLDM = BinaryLabelDatasetMetric(dataset_test_pred,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    
    dataset_test_pred.features = dataset_test.features 
    
    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
     
    name = ""
    if preprocessing_algo is not None:
        name += get_model_name(preprocessing_algo) + " + "
    if inprocessing_algo is not None:
        name += get_model_name(inprocessing_algo) + " + "
    if postprocessing_algo is not None:
        name += get_model_name(postprocessing_algo)
    if name == "":
        name = get_model_name(model)
        
    if name.endswith(" + "):
        lastIndex = name.rindex(" + ")
        name = name[:lastIndex]
    
    #print(run_classification_metrics(CM))
    #print(run_binary_dataset_metrics(BLDM))
    metrics = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))
       
    return {"key":name, "val":metrics}

In [64]:
#datasets = ["compas", "german", "adult", "bank"]
datasets = ["bank"]
for dataset_name in datasets:
    print("DATASET NAME: ", dataset_name)
    dataset, pro_attr, privileged_groups, unprivileged_groups, optim_options = get_dataset_options(dataset_name)
    
    preprocessing_algos = [OptimPreproc(OptTools, optim_options, unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups),
                           #preprocessing.DisparateImpactRemover(sensitive_attribute=pro_attr),
                           None,
                           #LFR(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, k=10, Ax=0.1, Ay=1.0, Az=2.0, verbose=0),
                          #preprocessing.Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),
                           ]
                          
    inprocessing_algos = [None,
                           #inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=seed),
                          GerryFairClassifier(),
                          inprocessing.PrejudiceRemover(sensitive_attr=pro_attr),
                          #inprocessing.ExponentiatedGradientReduction(LogisticRegression(), constraints="DemographicParity", drop_prot_attr=False),
                          #inprocessing.GridSearchReduction(LogisticRegression(), prot_attr=pro_attr, constraints="DemographicParity", drop_prot_attr=False),
                          ]

    postprocessing_algos = [None,
                            postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=0),
                            #postprocessing.CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=0),
                            postprocessing.RejectOptionClassification(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),  
                            ]

    df = {}
    all_data = {}

    for pre in preprocessing_algos:
        for inproc in inprocessing_algos:
            for post in postprocessing_algos:    
                kfold_data = []   # store data for each fold
                col_name = None
                # k-fold cross vaidation - Repeat the process 3 times
                for seed in [3, 11, 17]:  #[3, 5, 11, 17, 29]:
                    try:
                        res = execute_intervention(dataset, dataset_name, privileged_groups, unprivileged_groups, 
                                 preprocessing_algo=copy.deepcopy(pre), 
                                 inprocessing_algo=copy.deepcopy(inproc),
                                 postprocessing_algo=copy.deepcopy(post), seed=seed)
                        
                        col_name = res["key"]
                        if col_name:
                            kfold_data.append(res["val"])
                            all_data[col_name+" - "+str(seed)] = res["val"]
                        
                    except KeyboardInterrupt:
                        raise KeyboardInterrupt()
                    except Exception as e:
                        print("FAILED: " + get_model_name(pre) + ", " + get_model_name(inproc) + ", " + get_model_name(post) + " on dataset " + get_dataset_name(dataset), e)
                
                if col_name:
                    df[col_name] = np.array(kfold_data).mean(axis=0).tolist()
    
    df = pd.DataFrame.from_dict(df)
    all_data = pd.DataFrame.from_dict(all_data)
    df.index = all_data.index = ["Accuracy", "F1 Score", "Theil Index",
                    "False Positive Rate - Unprivileged", "False Positive Rate - Privileged",
                    "False Negative Rate - Unprivileged", "False Negative Rate - Privileged",
                    "Accuracy - Unprivileged", "Accuracy - Privileged",
                    "False Discovery Rate - Unprivileged", "False Discovery Rate - Privileged",
                    "False Omission Rate - Unprivileged", "False Omission Rate - Privileged",
                    "Num True Pos", "Num True Neg", "Num False Pos", "Num False Neg",
                    "Num True Pos - Privileged", "Num True Neg - Privileged", "Num False Pos - Privileged", "Num False Neg - Privileged",
                    "Num True Pos - Unprivileged", "Num True Neg - Unprivileged", "Num False Pos - Unprivileged", "Num False Neg - Unprivileged",
                    "F1 Score - Privileged", "F1 Score - Unprivileged",
                    "Privileged base Rate", "Unprivileged base Rate", "Consistency"]

    df = df.T
    all_data = all_data.T
    df.to_csv("./data/"+ dataset_name +".csv", sep=',', encoding='utf-8')
    all_data.to_csv("./data/All_Data -"+ dataset_name +".csv", sep=',', encoding='utf-8')
    display(df)        

DATASET NAME:  bank
3 OP None None
FAILED: OP, None, None on dataset Bank Dataset 'NoneType' object is not subscriptable
11 OP None None
FAILED: OP, None, None on dataset Bank Dataset 'NoneType' object is not subscriptable
17 OP None None
FAILED: OP, None, None on dataset Bank Dataset 'NoneType' object is not subscriptable
3 OP None EOP
FAILED: OP, None, EOP on dataset Bank Dataset 'NoneType' object is not subscriptable
11 OP None EOP
FAILED: OP, None, EOP on dataset Bank Dataset 'NoneType' object is not subscriptable
17 OP None EOP
FAILED: OP, None, EOP on dataset Bank Dataset 'NoneType' object is not subscriptable
3 OP None ROC
FAILED: OP, None, ROC on dataset Bank Dataset 'NoneType' object is not subscriptable
11 OP None ROC
FAILED: OP, None, ROC on dataset Bank Dataset 'NoneType' object is not subscriptable
17 OP None ROC
FAILED: OP, None, ROC on dataset Bank Dataset 'NoneType' object is not subscriptable
3 OP GFC None
FAILED: OP, GFC, None on dataset Bank Dataset 'NoneType' object

,Accuracy,F1 Score,Theil Index,False Positive Rate - Unprivileged,False Positive Rate - Privileged,False Negative Rate - Unprivileged,False Negative Rate - Privileged,Accuracy - Unprivileged,Accuracy - Privileged,False Discovery Rate - Unprivileged,...,Num False Neg - Privileged,Num True Pos - Unprivileged,Num True Neg - Unprivileged,Num False Pos - Unprivileged,Num False Neg - Unprivileged,F1 Score - Privileged,F1 Score - Unprivileged,Privileged base Rate,Unprivileged base Rate,Consistency
Logistic Regression,0.900467,0.518767,0.085833,0.072500,0.029800,0.523533,0.577533,0.824067,0.902733,0.334767,...,632.666667,28.000000,186.000000,14.666667,31.000000,0.516933,0.551100,0.078200,0.164200,0.951133
EOP,0.881033,0.478233,0.091867,0.065900,0.053833,0.620733,0.564767,0.807367,0.883167,0.366933,...,618.666667,22.333333,187.333333,13.333333,36.666667,0.478567,0.471633,0.100900,0.137300,0.919267
ROC,0.889467,0.332833,0.108733,0.021567,0.013433,0.720700,0.784500,0.818700,0.891533,0.191967,...,859.333333,16.333333,196.333333,4.333333,42.666667,0.328667,0.407633,0.038367,0.079633,0.968667
GFC,0.895367,0.446033,0.096067,0.046100,0.022900,0.612767,0.668967,0.823900,0.897467,0.282600,...,733.000000,22.666667,191.333333,9.333333,36.333333,0.443133,0.497533,0.060900,0.123067,0.972000
GFC + EOP,0.886600,0.431267,0.098667,0.041200,0.034333,0.664967,0.658900,0.816267,0.888667,0.286000,...,722.000000,19.666667,192.333333,8.333333,39.333333,0.430200,0.450733,0.072167,0.107833,0.955400
GFC + ROC,0.895333,0.446000,0.096100,0.046100,0.022933,0.606700,0.669267,0.825167,0.897400,0.279533,...,733.333333,23.000000,191.333333,9.333333,36.000000,0.442767,0.503000,0.060833,0.124333,0.972067
PR,0.900667,0.517733,0.086033,0.051000,0.029700,0.575833,0.577433,0.827567,0.902800,0.276633,...,632.666667,24.666667,190.333333,10.333333,34.333333,0.517233,0.522433,0.078133,0.134467,0.951600
PR + EOP,0.893300,0.501000,0.088467,0.037800,0.039033,0.609900,0.573467,0.830133,0.895100,0.238100,...,628.333333,22.666667,193.000000,7.666667,36.333333,0.500600,0.504767,0.086800,0.116467,0.938733
PR + ROC,0.891533,0.370433,0.104733,0.029833,0.016033,0.701967,0.748267,0.816033,0.893733,0.225867,...,819.333333,17.333333,194.666667,6.000000,41.666667,0.367667,0.417833,0.045167,0.089800,0.965167


<h3>EXTRA</h3>

In [41]:
scale_orig = StandardScaler()
g.features = scale_orig.fit_transform(g.features)
g.convert_to_dataframe()[0]

,sex,age,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,age_cat=25 - 45,age_cat=Greater than 45,age_cat=Less than 25,...,c_charge_desc=Viol Injunct Domestic Violence,c_charge_desc=Viol Injunction Protect Dom Vi,c_charge_desc=Viol Pretrial Release Dom Viol,c_charge_desc=Viol Prot Injunc Repeat Viol,c_charge_desc=Violation License Restrictions,c_charge_desc=Violation Of Boater Safety Id,c_charge_desc=Violation of Injunction Order/Stalking/Cyberstalking,c_charge_desc=Voyeurism,c_charge_desc=arrest case no charge,two_year_recid
1,0.0,2.939659,0.0,-0.127668,-0.183308,-0.235203,-0.684403,-1.156231,1.942478,-0.528640,...,-0.087634,-0.031207,-0.044155,-0.054105,-0.012735,-0.012735,-0.012735,-0.012735,-0.381633,0.0
3,0.0,-0.045361,0.0,-0.127668,-0.183308,-0.235203,-0.684403,0.864879,-0.514806,-0.528640,...,-0.087634,-0.031207,-0.044155,-0.054105,-0.012735,-0.012735,-0.012735,-0.012735,-0.381633,1.0
4,0.0,-0.898223,0.0,-0.127668,-0.183308,1.888511,0.158601,-1.156231,-0.514806,1.891645,...,-0.087634,-0.031207,-0.044155,-0.054105,-0.012735,-0.012735,-0.012735,-0.012735,-0.381633,1.0
7,0.0,0.807502,0.0,-0.127668,-0.183308,-0.235203,-0.684403,0.864879,-0.514806,-0.528640,...,-0.087634,-0.031207,-0.044155,-0.054105,-0.012735,-0.012735,-0.012735,-0.012735,-0.381633,0.0
8,0.0,0.551643,1.0,-0.127668,-0.183308,-0.235203,2.266112,0.864879,-0.514806,-0.528640,...,-0.087634,-0.031207,-0.044155,-0.054105,-0.012735,-0.012735,-0.012735,-0.012735,-0.381633,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10996,0.0,-0.983510,0.0,-0.127668,-0.183308,-0.235203,-0.684403,-1.156231,-0.514806,1.891645,...,-0.087634,-0.031207,-0.044155,-0.054105,-0.012735,-0.012735,-0.012735,-0.012735,-0.381633,0.0
10997,0.0,-0.983510,0.0,-0.127668,-0.183308,-0.235203,-0.684403,-1.156231,-0.514806,1.891645,...,-0.087634,-0.031207,-0.044155,-0.054105,-0.012735,-0.012735,-0.012735,-0.012735,-0.381633,0.0
10999,0.0,1.916224,0.0,-0.127668,-0.183308,-0.235203,-0.684403,-1.156231,1.942478,-0.528640,...,-0.087634,-0.031207,-0.044155,-0.054105,-0.012735,-0.012735,-0.012735,-0.012735,-0.381633,0.0
11000,1.0,-0.130647,0.0,-0.127668,-0.183308,-0.235203,-0.052150,0.864879,-0.514806,-0.528640,...,-0.087634,-0.031207,-0.044155,-0.054105,-0.012735,-0.012735,-0.012735,-0.012735,-0.381633,0.0


In [56]:
d = load_preproc_data_adult(['sex'])
d.convert_to_dataframe()[0]

,race,sex,Age (decade)=10,Age (decade)=20,Age (decade)=30,Age (decade)=40,Age (decade)=50,Age (decade)=60,Age (decade)=>=70,Education Years=6,Education Years=7,Education Years=8,Education Years=9,Education Years=10,Education Years=11,Education Years=12,Education Years=<6,Education Years=>12,Income Binary
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
48838,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
48839,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
48840,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
scale_orig = StandardScaler()
d.features = scale_orig.fit_transform(d.features)
d.convert_to_dataframe()[0]

,race,sex,Age (decade)=10,Age (decade)=20,Age (decade)=30,Age (decade)=40,Age (decade)=50,Age (decade)=60,Age (decade)=>=70,Education Years=6,Education Years=7,Education Years=8,Education Years=9,Education Years=10,Education Years=11,Education Years=12,Education Years=<6,Education Years=>12,Income Binary
0,-2.428701,1.0,-0.232754,1.751705,-0.600007,-0.530412,-0.395933,-0.258261,-0.144649,-0.171088,5.094580,-0.116769,-0.690988,-0.535289,-0.209896,-0.184093,-0.234702,-0.574182,0.0
1,0.411743,1.0,-0.232754,-0.570872,1.666646,-0.530412,-0.395933,-0.258261,-0.144649,-0.171088,-0.196287,-0.116769,1.447204,-0.535289,-0.209896,-0.184093,-0.234702,-0.574182,0.0
2,0.411743,1.0,-0.232754,1.751705,-0.600007,-0.530412,-0.395933,-0.258261,-0.144649,-0.171088,-0.196287,-0.116769,-0.690988,-0.535289,-0.209896,5.432051,-0.234702,-0.574182,1.0
3,-2.428701,1.0,-0.232754,-0.570872,-0.600007,1.885327,-0.395933,-0.258261,-0.144649,-0.171088,-0.196287,-0.116769,-0.690988,1.868149,-0.209896,-0.184093,-0.234702,-0.574182,1.0
4,0.411743,0.0,4.296390,-0.570872,-0.600007,-0.530412,-0.395933,-0.258261,-0.144649,-0.171088,-0.196287,-0.116769,-0.690988,1.868149,-0.209896,-0.184093,-0.234702,-0.574182,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.411743,0.0,-0.232754,1.751705,-0.600007,-0.530412,-0.395933,-0.258261,-0.144649,-0.171088,-0.196287,-0.116769,-0.690988,-0.535289,-0.209896,5.432051,-0.234702,-0.574182,0.0
48838,0.411743,1.0,-0.232754,-0.570872,-0.600007,1.885327,-0.395933,-0.258261,-0.144649,-0.171088,-0.196287,-0.116769,1.447204,-0.535289,-0.209896,-0.184093,-0.234702,-0.574182,1.0
48839,0.411743,0.0,-0.232754,-0.570872,-0.600007,-0.530412,2.525680,-0.258261,-0.144649,-0.171088,-0.196287,-0.116769,1.447204,-0.535289,-0.209896,-0.184093,-0.234702,-0.574182,0.0
48840,0.411743,1.0,-0.232754,1.751705,-0.600007,-0.530412,-0.395933,-0.258261,-0.144649,-0.171088,-0.196287,-0.116769,1.447204,-0.535289,-0.209896,-0.184093,-0.234702,-0.574182,0.0


In [63]:
# Individual Testing
dataset_name = "bank"
dataset, pro_attr, privileged_groups, unprivileged_groups, optim_options = get_dataset_options(dataset_name)

#print("Specialized function: ", dataset_name)
#dataset = get_OP_dataset(dataset_name) 

scale_orig = StandardScaler()
dataset.features = scale_orig.fit_transform(dataset.features)

dataset_train, dataset_test = dataset.split([0.7], shuffle = True, seed=7)

dataset_train_pred = dataset_train.copy(deepcopy=True)
dataset_test_pred = dataset_test.copy(deepcopy=True)

#pre = LFR(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, k=10, Ax=0.1, Ay=1.0, Az=2.0, verbose=0)
#pre = preprocessing.DisparateImpactRemover(sensitive_attribute=pro_attr)
pre = OptimPreproc(OptTools, optim_options, unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups)
pre.fit(dataset_train_pred)
dataset_train_pred = pre.transform(dataset_train_pred)
dataset_train_pred = dataset_train.align_datasets(dataset_train_pred)
dataset_test_pred = pre.transform(dataset_test_pred)
dataset_test_pred = dataset_test.align_datasets(dataset_test_pred)


X_train = dataset_train_pred.features
y_train = dataset_train_pred.labels #.ravel()
model = LogisticRegression()  
model.fit(X_train, y_train)
fav_idx = np.where(model.classes_ == dataset_train.favorable_label)[0][0]
dataset_train_pred.scores = model.predict_proba(dataset_train_pred.features)[:,fav_idx].reshape(-1,1) 
dataset_train_pred.labels = model.predict(dataset_train_pred.features).reshape(-1,1)
dataset_test_pred.scores = model.predict_proba(dataset_test_pred.features)[:,fav_idx].reshape(-1,1) 
dataset_test_pred.labels = model.predict(dataset_test_pred.features).reshape(-1,1)     


#inp = inprocessing.GridSearchReduction(LogisticRegression(), prot_attr=pro_attr, constraints="DemographicParity", drop_prot_attr=False)
#inp = inprocessing.PrejudiceRemover(sensitive_attr=pro_attr)
#inp = GerryFairClassifier()
#inp.fit(dataset_train_pred)


#dataset_train_pred.scores = inp.predict(dataset_train_pred, threshold=None).labels
#dataset_test_pred.scores = inp.predict(dataset_test_pred, threshold=None).labels
#dataset_train_pred.labels = inp.predict(dataset_train_pred, threshold=0.5).labels
#dataset_test_pred.labels = inp.predict(dataset_test_pred, threshold=0.5).labels


#dataset_train_pred = inp.predict(dataset_train_pred)
#dataset_test_pred = inp.predict(dataset_test_pred)

'''
y_train_pred = np.zeros_like(dataset_train_pred.scores)
y_train_pred[dataset_train_pred.scores >= class_thresh] = dataset_train_pred.favorable_label
y_train_pred[~(dataset_train_pred.scores >= class_thresh)] = dataset_train_pred.unfavorable_label
dataset_train_pred.labels = y_train_pred

y_test_pred = np.zeros_like(dataset_test_pred.scores)
y_test_pred[dataset_test_pred.scores >= class_thresh] = dataset_test_pred.favorable_label
y_test_pred[~(dataset_test_pred.scores >= class_thresh)] = dataset_test_pred.unfavorable_label
dataset_test_pred.labels = y_test_pred
#dataset_test_pred.labels = y_test_pred#dataset_train_pred.features = dataset_train.features
'''

#pp = postprocessing.CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=7)
#pp = postprocessing.RejectOptionClassification(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
#pp = postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=0)
#dataset_train_pred.features = dataset_train.features
#pp = pp.fit(dataset_train, dataset_train_pred)
#dataset_test_pred = pp.predict(dataset_test_pred)


BLDM = BinaryLabelDatasetMetric(dataset_test_pred,
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)


dataset_test_pred.features = dataset_test.features 
CM = ClassificationMetric(dataset_test,
                          dataset_test_pred,
                          unprivileged_groups=unprivileged_groups,
                          privileged_groups=privileged_groups)
    

print(run_classification_metrics(CM))
print(run_binary_dataset_metrics(BLDM))

TypeError: 'NoneType' object is not subscriptable

In [25]:
np.sum(dataset_train_pred.labels != dataset_train.labels)

1283

In [186]:
dataset_test_pred.scores

array([[0.88888889],
       [1.        ],
       [1.        ],
       ...,
       [0.88888889],
       [0.88888889],
       [0.88888889]])

In [165]:
CM.binary_confusion_matrix()

{'TP': 3353.0, 'FP': 10214.0, 'TN': 0.0, 'FN': 0.0}

In [121]:
dataset_train_pred = inp.predict(dataset_train_pred)
dataset_test_pred = inp.predict(dataset_test_pred) 

In [107]:
dataset_test_pred.labels.ravel()

array([1, 1, 1, ..., 1, 1, 1])

In [15]:
#dataset_train.metadata  #.labels.ravel()
CM.binary_confusion_matrix()

{'TP': 3330.0, 'FP': 0.0, 'TN': 10237.0, 'FN': 0.0}

In [108]:
from sklearn.metrics import accuracy_score
accuracy_score(dataset_test.labels, dataset_test_pred.labels)

0.7566666666666667

In [79]:
dataset_test_pred.labels = dataset_test_pred.labels.astype('float64') #.ravel()

In [86]:
dataset_test_pred.labels

array([1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0.

In [87]:
dataset_test.labels

array([1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0.,
       0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
       1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0.,
       1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
       0., 0., 0., 0., 0.

In [88]:
#type(dataset_test_pred.labels[0][0])
type(dataset_test)

aif360.datasets.german_dataset.GermanDataset

In [54]:
dataset = GermanDataset()
dataset.labels = dataset.labels-1
inp = inprocessing.GridSearchReduction(LogisticRegression(), prot_attr=pro_attr, constraints="DemographicParity", drop_prot_attr=False)
inp.fit(dataset)